# Kita Pisahkan datanya menjadi 3

- train (data latih)
- validation (validasi untuk data latih)
- test (data testing untuk menguji model)

In [1]:
# extract melalui notebook jika diperlukan
# !unzip flowers-recognition.zip

In [38]:
import os
import pandas as pd

In [39]:
mypath= 'all_augmented/'

In [40]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(mypath):
    for name in files:
        full_path.append(os.path.join(path, name).replace("\\","/")) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [41]:
# memasukan variabel yang sudah dikumpulkan pada looping di atas menjadi sebuah dataframe agar rapih
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

tag
Keriting    167
Kuning      173
Layu        160
dtype: int64

In [42]:
#cek sample datanya
df.head()

,path,file_name,tag
0,all_augmented/Keriting/IMG20241127171041_01.jpg,IMG20241127171041_01.jpg,Keriting
1,all_augmented/Keriting/IMG20241127171042.jpg,IMG20241127171042.jpg,Keriting
2,all_augmented/Keriting/IMG20241127171043.jpg,IMG20241127171043.jpg,Keriting
3,all_augmented/Keriting/IMG20241127171043_01.jpg,IMG20241127171043_01.jpg,Keriting
4,all_augmented/Keriting/IMG20241127171053.jpg,IMG20241127171053.jpg,Keriting


# Train test split

In [43]:
#load library untuk train test split
from sklearn.model_selection import train_test_split

In [44]:
#variabel yang digunakan pada pemisahan data ini
X= df['path']
y= df['tag']

In [45]:
# split dataset awal menjadi data train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=300)

In [46]:
# kemudian data test dibagi menjadi 2 sehingga menjadi data test dan data validation.
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=100)

In [47]:
# menyatukan kedalam masing-masing dataframe

df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

In [48]:
print('train size', len(df_tr))
print('val size', len(df_te))
print('test size', len(df_val))

train size 400
val size 50
test size 50


In [49]:
# Menggabungkan data frame df_tr, df_te, dan df_val
df_all = pd.concat([df_tr, df_te, df_val], ignore_index=True)

print('===================================================== \n')
print(df_all.groupby(['set', 'tag']).size(), '\n')

print('===================================================== \n')

# Cek sample datanya
df_all.sample(3)



set         tag     
test        Keriting     17
            Kuning       18
            Layu         15
train       Keriting    129
            Kuning      141
            Layu        130
validation  Keriting     21
            Kuning       14
            Layu         15
dtype: int64 




,path,tag,set
254,all_augmented/Layu/IMG_20241025_114537.jpg,Layu,train
230,all_augmented/Kuning/IMG_20241025_102929774_BU...,Kuning,train
142,all_augmented/Kuning/IMG_20241025_103505_877.jpg,Kuning,train


# Merapihkan ke folder set masing-masing

In [50]:
# menghapus folder dataset jika diperlukan
#!rm -rf dataset/

In [51]:
import shutil
from tqdm.notebook import tqdm as tq

In [52]:
datasource_path = "all_augmented/"
dataset_path = "all_augmented/after_split/"

In [53]:
for index, row in tq(df_all.iterrows(), ):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

# Finish! Dataset siap digunakan 

In [62]:
import os
import numpy as np
import uuid
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import random

# === Konfigurasi Dataset ===
original_dataset_path = 'all_augmented/after_split/validation/Layu'  # Path folder dataset asli
target_size = (224, 224)  # Ukuran target gambar
augmentations_per_image = 6  # Jumlah augmentasi yang dilakukan per gambar

# === Fungsi untuk Augmentasi ===
def save_augmented_image(img_array, category_path, prefix):
    """
    Simpan gambar hasil augmentasi ke folder yang sama.
    """
    img_array = np.clip(img_array, 0, 255).astype('uint8')  # Jaga nilai pixel tetap valid
    img = array_to_img(img_array)  # Konversi kembali ke format gambar
    save_path = os.path.join(category_path, f"{prefix}_{uuid.uuid4()}.jpg")
    img.save(save_path)

def augment_image(img_array):
    """
    Terapkan berbagai augmentasi pada gambar.
    """
    augmented_images = []

    # Flip Horizontal
    augmented_images.append(("flip", np.fliplr(img_array)))

    # Rotate Clockwise 90°
    augmented_images.append(("rot90_cw", np.rot90(img_array, k=-1)))

    # Rotate Counter-Clockwise 90°
    augmented_images.append(("rot90_ccw", np.rot90(img_array, k=1)))

    # Random Brightness
    brightness_factor = random.uniform(0.8, 1.2)
    augmented_images.append(("bright", img_array * brightness_factor))

    # Random Exposure Adjustment
    exposure_factor = random.uniform(0.9, 1.1)
    augmented_images.append(("expose", img_array * exposure_factor))

    # Random Rotation (-15° to +15°)
    rotation_factor = random.choice([-15, 15])
    augmented_images.append((f"rot{rotation_factor}", np.rot90(img_array, k=rotation_factor)))

    return augmented_images

def process_folder(folder_path):
    """
    Proses semua gambar di dalam folder, menyimpan augmentasi ke folder yang sama.
    """
    total_original_images = 0
    total_augmented_images = 0

    for root, _, files in os.walk(folder_path):
        for img_file in files:
            if not img_file.lower().endswith(('jpg', 'jpeg', 'png')):
                continue  # Skip jika bukan file gambar

            # Path ke gambar
            img_path = os.path.join(root, img_file)

            # Load gambar asli
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)

            # Terapkan augmentasi
            augmented_images = augment_image(img_array)
            for prefix, augmented_img in random.sample(augmented_images, augmentations_per_image):
                save_augmented_image(augmented_img, root, prefix)  # Simpan di folder yang sama
                total_augmented_images += 1

            total_original_images += 1

    return total_original_images, total_augmented_images

# === Jalankan Augmentasi ===
total_original_images, total_augmented_images = process_folder(original_dataset_path)

print(f"Proses augmentasi selesai.")
print(f"Total gambar asli diproses: {total_original_images}")
print(f"Total gambar augmentasi yang dihasilkan: {total_augmented_images}")


Proses augmentasi selesai.
Total gambar asli diproses: 15
Total gambar augmentasi yang dihasilkan: 90
